In [99]:
from google.colab import drive
import numpy as np

drive.mount('/content/drive')

data = np.load("/content/drive/MyDrive/CV/dataset.npz")

train_images = data["train_images"]
train_labels = data["train_labels"]
test_images  = data["test_images"]
test_labels  = data["test_labels"]


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [100]:
import cv2
import numpy as np
from tqdm import tqdm

def enhance_image(img):
    # 1️⃣ Chuyển sang YUV và cân bằng histogram nhẹ hơn
    img_yuv = cv2.cvtColor((img * 255).astype(np.uint8), cv2.COLOR_RGB2YUV)
    y = img_yuv[:, :, 0]
    # Cân bằng histogram có trọng số, không để quá sáng
    y_eq = cv2.equalizeHist(y)
    img_yuv[:, :, 0] = cv2.addWeighted(y, 0.7, y_eq, 0.3, 0)  # 70% gốc + 30% cân bằng
    img = cv2.cvtColor(img_yuv, cv2.COLOR_YUV2RGB)

    # 2️⃣ Sharpen nhẹ
    gaussian = cv2.GaussianBlur(img, (3, 3), 0)
    sharpened = cv2.addWeighted(img, 1.1, gaussian, -0.1, 0)  # Giảm cường độ sharpen

    # 3️⃣ Tăng cường biên vừa phải
    gray = cv2.cvtColor(sharpened, cv2.COLOR_RGB2GRAY)
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
    edges = cv2.magnitude(sobelx, sobely)
    edges = cv2.normalize(edges, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    edges_colored = cv2.cvtColor(edges, cv2.COLOR_GRAY2RGB)

    # 4️⃣ Trộn nhẹ với ảnh gốc
    enhanced = cv2.addWeighted(sharpened, 0.97, edges_colored, 0.03, 0)

    # 5️⃣ Chuẩn hóa [0, 1]
    enhanced = enhanced.astype("float32") / 255.0
    return enhanced

# Áp dụng cho toàn bộ tập train
train_images_enhanced = np.zeros_like(train_images)
for i in tqdm(range(len(train_images))):
    train_images_enhanced[i] = enhance_image(train_images[i])



100%|██████████| 39209/39209 [00:14<00:00, 2735.08it/s]


In [ ]:
import matplotlib.pyplot as plt

idx = np.random.randint(0, len(train_images))
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plt.imshow(train_images[idx])
plt.title("Ảnh gốc sau tiền xử lý")

plt.subplot(1,2,2)
plt.imshow(train_images_enhanced[idx])
plt.title("Ảnh sau tăng cường (enhanced)")
plt.show()


In [126]:
import cv2
import numpy as np
from skimage.measure import shannon_entropy

def image_quality_metrics(img):
    gray = cv2.cvtColor((img * 255).astype(np.uint8), cv2.COLOR_RGB2GRAY)
    sharpness = cv2.Laplacian(gray, cv2.CV_64F).var()
    contrast = np.std(gray)
    entropy = shannon_entropy(gray)
    return sharpness, contrast, entropy

# So sánh 1 ảnh gốc và 1 ảnh enhanced
sharp_orig, cont_orig, ent_orig = image_quality_metrics(train_images[0])
sharp_enh, cont_enh, ent_enh = image_quality_metrics(train_images_enhanced[0])

print("GỐC:")
print(f"  Sharpness: {sharp_orig:.2f}, Contrast: {cont_orig:.2f}, Entropy: {ent_orig:.2f}")
print("TĂNG CƯỜNG:")
print(f"  Sharpness: {sharp_enh:.2f}, Contrast: {cont_enh:.2f}, Entropy: {ent_enh:.2f}")


GỐC:
  Sharpness: 13.51, Contrast: 10.96, Entropy: 4.84
TĂNG CƯỜNG:
  Sharpness: 147.94, Contrast: 27.43, Entropy: 6.05


In [131]:
sharp_orig_list, cont_orig_list, ent_orig_list = [], [], []
sharp_enh_list, cont_enh_list, ent_enh_list = [], [], []

for i in range(39209):  # ví dụ tính trên 100 ảnh đầu tiên
    s, c, e = image_quality_metrics(train_images[i])
    sharp_orig_list.append(s)
    cont_orig_list.append(c)
    ent_orig_list.append(e)

    s, c, e = image_quality_metrics(train_images_enhanced[i])
    sharp_enh_list.append(s)
    cont_enh_list.append(c)
    ent_enh_list.append(e)

print("Trung bình GỐC:", np.mean(sharp_orig_list), np.mean(cont_orig_list), np.mean(ent_orig_list))
print("Trung bình TĂNG CƯỜNG:", np.mean(sharp_enh_list), np.mean(cont_enh_list), np.mean(ent_enh_list))


Trung bình GỐC: 513.8207754693286 40.873540044072804 6.082146368938843
Trung bình TĂNG CƯỜNG: 689.8163855095249 48.454680156350626 6.826223905239407
